## Experiment on number of layers and pL

In [ ]:
import tensorflow as tf
import numpy as np
from utils.neuralnets.StochasticNet import StochasticNet
from utils.trainer import train
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D,Input,Flatten,Dense
from tensorflow.keras import Model
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [2]:
data=tfds.load('svhn_cropped')
data

{'train': <PrefetchDataset shapes: {image: (32, 32, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 'test': <PrefetchDataset shapes: {image: (32, 32, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 'extra': <PrefetchDataset shapes: {image: (32, 32, 3), label: ()}, types: {image: tf.uint8, label: tf.int64}>}

In [3]:
def train_val_split(data1,data2,number_of_samples=[400,100]):
    val = []
    val_label = []
    train = []
    train_label =[]
    label_count = np.zeros(10)
    for sample in data1:
        if label_count[sample['label']] < number_of_samples[0]:
            val.append(sample['image'].numpy().astype('float32'))
            val_label.append(sample['label'].numpy().astype('float32'))
            label_count[sample['label']] += 1
        else:
            train.append(sample['image'].numpy().astype('float32'))
            train_label.append(sample['label'].numpy().astype('float32'))
    label_count = np.zeros(10)
    extra_count = 0
    for sample in data2:
        if label_count[sample['label']] < number_of_samples[1]:
            val.append(sample['image'].numpy().astype('float32'))
            val_label.append(sample['label'].numpy().astype('float32'))
            label_count[sample['label']] += 1
        else:
            train.append(sample['image'].numpy().astype('float32'))
            train_label.append(sample['label'].numpy().astype('float32'))
        extra_count += 1
        if extra_count > 30000:
            break
    return (np.array(train),np.array(train_label)),(np.array(val),np.array(val_label))

In [4]:
train_data, val_data = train_val_split(data['train'],data['extra'])
x_train ,y_train = train_data
x_val, y_val = val_data
x_test = np.array([sample['image'].numpy().astype('float32') for sample in data['test']])
y_test = np.array([sample['label'].numpy().astype('float32') for sample in data['test']])

In [5]:
depth = [20,38,56,74,92]
prob = [0.5,0.6,0.7,0.8,0.9,1]

In [6]:
input_shape = x_train.shape[1:]
num_class = 10
batch_size = 128
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
validation_data = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)

In [7]:
results = []

for d in depth:
    for p in prob:
        model = StochasticNet(input_shape=input_shape,num_class=num_class,layers=d, p_L=p)
        result = train(model,d,p,train_data,validation_data,x_test,y_test)
        results.append(result)
        file = open('./Accuracies/acc', 'a')
        file.write('{}\t{}\n'.format(result[1],result[2]))
        file.close()
        np.save("./Logs/StochasticNet%d_%1.2f"%(d,p),result[0])


Epoch 1/24
Training loss at step 0: 8.9014
Training loss at step 200: 2.3360
Training loss at step 400: 2.2421
Training loss at step 600: 2.2826
Training accuracy: 0.1877 Validation accuracy: 0.1286 Time taken: 233.44s
Epoch 2/24
Training loss at step 0: 2.1879
Training loss at step 200: 1.4141
Training loss at step 400: 0.8798
Training loss at step 600: 0.4570
Training accuracy: 0.6419 Validation accuracy: 0.8046 Time taken: 218.96s
Epoch 3/24
Training loss at step 0: 4.2538
Training loss at step 200: 0.4328
Training loss at step 400: 0.4018
Training loss at step 600: 0.1841
Training accuracy: 0.8861 Validation accuracy: 0.8756 Time taken: 218.36s
Epoch 4/24
Training loss at step 0: 3.9273
Training loss at step 200: 0.2743
Training loss at step 400: 0.3195
Training loss at step 600: 0.0760
Training accuracy: 0.9218 Validation accuracy: 0.8872 Time taken: 217.93s
Epoch 5/24
Training loss at step 0: 3.4804
Training loss at step 200: 0.2081
Training loss at step 400: 0.2838
Training loss